<a href="https://colab.research.google.com/github/rnop/nmr_tournament/blob/main/introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction to the Numerai Tournament
Numerai is a hedge fund that trades the global markets based on models created by data scientists all over the world. Numerai is unique in that it provides free high-quality financial datasets that are worth millions of dollars to any user wanting to participate in their tournament. Users are able to build their own models on this anonymized and obfuscated dataset, submit their predictions, and follow their investment performance on the live stock market. If users are confident about their models, they are able to stake on them with real money using Numerai's cryptocurrency, Numeraire (NMR).

### About this Notebook
The purpose of this notebook is to provide an introduction on how to approach the main Numerai tournament.

What's included:
* how to read in the Numerai data via API
* approaches to dimensionality reduction
* training an xgboost model 
* bayesian optimization techniques
* calculating predictions from the current round

### Disclaimer
**This model is not guaranteed to make you money.** I am currently not staking this particular model in the tournament. This notebook only serves to provide you an introduction to the tournament and to give some of my personal input on how to tackle this data science problem. 

In [1]:
# Download the numerai library 
! pip install numerapi

In [2]:
import pandas as pd
import numpy as np
import numerapi

### Import data
* Training data contains 501,808 observations
* Tournament data contains the testing and validation sets, and the live observations you need to predict on for the upcoming round

In [3]:
host = 'numerai-public-datasets.s3-us-west-2.amazonaws.com'
train_filename = 'latest_numerai_training_data.csv.xz'
tourney_filename = 'latest_numerai_tournament_data.csv.xz'

train_df = pd.read_csv('https://{}/{}'.format(host, train_filename))
tourney_df = pd.read_csv('https://{}/{}'.format(host, tourney_filename))

#Confirm round number
napi = numerapi.NumerAPI(verbosity="info")
current_round = napi.get_current_round()
print()
print("ROUND NUMBER: ", current_round)


ROUND NUMBER:  260


In [4]:
train_df.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,...,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,0.25,0.75,0.75,0.25,0.25,1.00,0.75,0.5,1.0,0.5,0.00,0.50,0.50,0.00,0.0,0.00,1.00,0.25,0.00,0.50,0.25,0.75,0.50,1.00,0.75,0.75,0.5,0.50,0.75,0.5,...,0.75,0.75,0.75,0.50,1.00,1.00,0.50,0.75,0.5,0.25,0.25,0.75,0.50,1.00,0.5,0.75,0.75,0.25,0.50,1.00,0.75,0.50,0.50,1.00,0.25,0.5,0.50,0.50,0.75,1.00,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75,0.50
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,0.25,0.50,0.50,0.00,0.50,0.00,0.50,0.5,0.5,0.5,0.50,0.25,0.25,0.50,0.0,1.00,0.50,0.50,0.50,0.75,0.50,0.50,0.75,0.25,0.50,0.75,0.5,0.25,0.75,0.5,...,0.25,0.25,0.25,1.00,1.00,0.50,0.50,0.50,0.0,0.25,1.00,0.50,1.00,1.00,0.5,0.50,0.50,1.00,0.25,0.75,1.00,0.25,0.25,1.00,0.50,0.5,0.50,0.75,0.75,0.75,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00,0.25
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,0.25,0.00,0.25,0.50,1.00,0.50,0.75,0.5,0.5,1.0,0.50,0.50,0.50,0.25,0.0,0.25,0.75,0.75,0.75,0.50,0.75,0.50,0.25,0.50,0.75,0.25,0.5,0.50,0.75,0.5,...,0.25,1.00,1.00,1.00,0.50,1.00,1.00,1.00,0.5,1.00,0.00,1.00,1.00,0.50,1.0,0.75,1.00,0.00,0.50,0.75,0.00,1.00,0.50,0.50,0.75,1.0,0.75,1.00,0.25,0.50,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75,0.25
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,0.75,0.25,0.50,0.50,0.50,0.75,0.50,1.0,0.5,0.5,0.00,1.00,0.00,0.75,0.0,0.50,0.50,0.50,0.50,0.00,0.50,0.50,0.75,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,1.00,1.00,0.75,0.75,1.00,0.75,0.75,0.75,1.0,0.75,1.00,0.75,1.00,0.75,1.0,0.00,0.50,0.75,1.00,0.75,1.00,0.75,1.00,1.00,0.00,0.5,0.75,0.75,1.00,0.75,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00,0.25
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,0.25,0.25,0.50,0.25,0.25,0.75,0.50,0.0,0.5,0.5,0.25,0.00,0.50,0.00,0.5,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,1.00,0.25,0.75,1.00,0.75,0.00,0.00,0.75,0.5,1.00,0.50,0.75,0.25,0.50,0.0,0.50,0.50,0.50,0.75,0.75,0.50,0.75,0.25,0.75,0.50,0.5,0.25,0.25,0.75,0.50,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75,0.75


In [5]:
tourney_df.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,...,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.0,0.75,0.5,0.25,0.50,0.5,0.25,0.0,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.0,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,...,0.00,0.0,0.25,0.5,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.5,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.5,1.0,0.00,0.00,0.25
1,n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.5,0.00,0.0,0.75,0.25,0.0,0.75,0.5,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.0,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,...,0.50,0.5,0.25,1.0,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.5,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.5,0.5,0.50,0.50,0.50
2,n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.0,1.00,1.0,1.00,0.50,0.5,1.00,1.0,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.5,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,...,0.25,0.5,0.00,0.0,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.0,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.0,0.5,0.50,0.00,1.00
3,n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.0,0.00,0.0,0.50,0.75,1.0,0.75,0.0,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.0,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,...,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.0,0.0,0.25,0.25,0.50
4,n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.0,0.50,0.5,0.25,0.25,0.5,0.50,1.0,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.0,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,...,0.00,0.5,0.50,0.0,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.0,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.0,0.0,0.25,1.00,0.75


In [6]:
# Number of observations in each dataset type
tourney_df['data_type'].value_counts()

test          1555267
validation     137779
live             5435
Name: data_type, dtype: int64

### Data Preprocessing
Most of the data cleaning has been done by Numerai in order to anonymize and obfuscate the data to us. This is done purposefully because of the data sharing rights from the data vendors Numerai spends millions of dollars on (thank you Numerai!). 

Steps:
* Extract features
* Convert era from string to integers

In [7]:
# Extract features
tourney_ids = tourney_df['id']
features = [c for c in tourney_df if c.startswith('feature')]

# The training data is also grouped into 120 different eras (1-120)
train_df["erano"] = train_df["era"].str.slice(3).astype(int)

valid_df = tourney_df[tourney_df['data_type']=='validation']
valid_df["erano"] = valid_df["era"].str.slice(3).astype(int)

# Extract eras
eras = train_df["erano"]
target = "target"

print("Training:", train_df.shape)
print("Validation:", valid_df.shape)
print("Tournament:", tourney_df.shape)

Training: (501808, 315)
Validation: (137779, 315)
Tournament: (1698481, 314)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [8]:
print("First five features:")
print(features[:5])
print()
print("Number of unique eras in training data: ", len(set(eras)))

First five features:
['feature_intelligence1', 'feature_intelligence2', 'feature_intelligence3', 'feature_intelligence4', 'feature_intelligence5']

Number of unique eras in training data:  120


### Split the training data into training/testing sets
Things to think about:
* Is it useful to use all of the features?
* How should we think about the different eras in the data?
* Do some features matter more in particular eras than in other eras?

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df[features], train_df[target],
                                                    stratify=train_df['erano'],
                                                    test_size=0.25, random_state=0)

print("X_train size: ", X_train.shape)
print("X_test_size: ", X_test.shape)

X_train size:  (376356, 310)
X_test_size:  (125452, 310)


Here, I decided to stratify the training dataset based on era ( "stratify=train_df['erano']" ). This allows the model to be trained on observations from all eras.





### Dimensionality Reduction Techniques

* PCA
* K-Means Clustering
* Autoencoder


PCA
* You can change the number of components or % variance retained

In [10]:
from sklearn.decomposition import PCA

# You can specify the number of components
pca = PCA(n_components=120)
pca_train = pca.fit_transform(X_train)
explained_variance = pca.explained_variance_ratio_
print(np.cumsum(explained_variance))

[0.10418853 0.18438231 0.23781067 0.27816663 0.31218791 0.34198411
 0.36653258 0.38880232 0.40842065 0.42659024 0.44218313 0.45664365
 0.47012225 0.48294476 0.49506729 0.50705258 0.51851441 0.52949213
 0.54032917 0.55054991 0.56046169 0.57020912 0.57954533 0.58874189
 0.59757115 0.60587842 0.61391695 0.62189413 0.62958392 0.63701823
 0.64435391 0.65151837 0.65841186 0.66521867 0.67182625 0.67840565
 0.68468038 0.69087394 0.69692735 0.70279109 0.70852448 0.71415297
 0.71968002 0.72509362 0.73040029 0.73557974 0.74063086 0.7455087
 0.75029002 0.7550304  0.75968773 0.76425439 0.76874844 0.77320189
 0.77752846 0.78179288 0.78598475 0.79013675 0.79415591 0.79816101
 0.80210181 0.8059548  0.80971857 0.81343191 0.817077   0.82065529
 0.82411799 0.82754494 0.83093963 0.83425688 0.83753016 0.840792
 0.84399386 0.84716242 0.85028036 0.85329643 0.85628105 0.85919797
 0.86204967 0.86481714 0.86754756 0.87024045 0.87286877 0.87547882
 0.87806247 0.88060077 0.88309165 0.88553618 0.88794799 0.8902863

In [11]:
# You could also specify the % of variance you want to retain
pca = PCA(n_components=0.90)
pca_train = pca.fit_transform(X_train)

# Explained variance of each component
explained_variance = pca.explained_variance_ratio_
print("Number of components with % variance: ", len(explained_variance))
print(np.cumsum(explained_variance))

Number of components with % variance:  95
[0.10418853 0.18438231 0.23781067 0.27816663 0.31218791 0.34198411
 0.36653258 0.38880232 0.40842065 0.42659024 0.44218313 0.45664365
 0.47012225 0.48294476 0.49506729 0.50705258 0.51851441 0.52949213
 0.54032917 0.55054991 0.56046169 0.57020912 0.57954533 0.58874189
 0.59757115 0.60587842 0.61391695 0.62189414 0.62958392 0.63701823
 0.64435391 0.65151837 0.65841186 0.66521867 0.67182625 0.67840565
 0.68468038 0.69087394 0.69692735 0.7027911  0.70852448 0.71415298
 0.71968003 0.72509363 0.7304003  0.73557974 0.74063087 0.7455087
 0.75029003 0.75503041 0.75968774 0.76425441 0.76874846 0.77320192
 0.77752848 0.78179291 0.78598479 0.79013678 0.79415595 0.79816105
 0.80210185 0.80595484 0.80971862 0.81343196 0.81707707 0.82065537
 0.82411808 0.82754506 0.83093977 0.83425703 0.83753036 0.84079221
 0.84399408 0.84716267 0.85028064 0.85329674 0.85628139 0.85919834
 0.86205015 0.86481775 0.86754834 0.87024136 0.87286978 0.87547995
 0.87806388 0.8806022

In [12]:
print("Original X_train shape:", X_train.shape)
print("PCA Transformed X_train shape:", pca_train.shape)

Original X_train shape: (376356, 310)
PCA Transformed X_train shape: (376356, 95)


K-Means Clustering
* You can change the number of clusters

In [13]:
from sklearn.cluster import KMeans, MiniBatchKMeans

kmeans120 = MiniBatchKMeans(n_clusters=120, random_state=6).fit(X_train)
kmeans120_train = kmeans120.transform(X_train)

In [14]:
print("Original X_train shape:", X_train.shape)
print("K-Means Clustered X_train shape:", kmeans120_train.shape)

Original X_train shape: (376356, 310)
K-Means Clustered X_train shape: (376356, 120)


Autoencoder
* You can change hidden_layer_size and # of epochs to train

In [27]:
import tensorflow as tf
from tensorflow import keras

def nn_autoencoder(train_data, hidden_layer_size=120, epochs=10):
    input_layer = keras.layers.Flatten(input_shape=(train_data.shape[1],))
    hidden_layer = keras.layers.Dense(hidden_layer_size, activation='relu')
    output_layer = keras.layers.Dense(train_data.shape[1])
    model = keras.Sequential([input_layer, hidden_layer, output_layer])
    model.compile('sgd', loss=tf.keras.losses.MeanSquaredError())
    model.fit(train_data, train_data, epochs=epochs)
    model2 = keras.Sequential([input_layer, 
                          keras.layers.Dense(hidden_layer_size, activation='sigmoid', weights=model.layers[1].get_weights())
                          ])
    return model2

autoencode_reduction = 240
autoencoder = nn_autoencoder(X_train, hidden_layer_size=autoencode_reduction, epochs=8)

Epoch 1/8
11762/11762 [==============================] - 16s 1ms/step - loss: 0.1523
Epoch 2/8
11762/11762 [==============================] - 16s 1ms/step - loss: 0.0914
Epoch 3/8
11762/11762 [==============================] - 16s 1ms/step - loss: 0.0760
Epoch 4/8
11762/11762 [==============================] - 16s 1ms/step - loss: 0.0668
Epoch 5/8
11762/11762 [==============================] - 16s 1ms/step - loss: 0.0602
Epoch 6/8
11762/11762 [==============================] - 17s 1ms/step - loss: 0.0553
Epoch 7/8
11762/11762 [==============================] - 16s 1ms/step - loss: 0.0514
Epoch 8/8
11762/11762 [==============================] - 16s 1ms/step - loss: 0.0481


In [28]:
autoenc_X_train = autoencoder.predict(X_train)
print("Original X_train shape:", X_train.shape)
print("Autoencoded X_train shape:", autoenc_X_train.shape)

Original X_train shape: (376356, 310)
Autoencoded X_train shape: (376356, 240)


### Bayesian Optimization for xgboost
* I am going to use the autoencoded training data but you can easily switch it to PCA or K-means clustering
* set 'tree_method':'gpu_hist' to train on GPU for faster training

In [29]:
# Download bayesian-optimization library
! pip install bayesian-optimization

In [30]:
import xgboost
from bayes_opt import BayesianOptimization

# Convert autoencoded training data to DMatrix for XGBoost
dtrain = xgboost.DMatrix(autoenc_X_train, y_train)
 
def bo_tune_xgb(max_depth, gamma, learning_rate, subsample, colsample_bytree, min_child_weight, n_estimators, alpha, eta):
    params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'learning_rate': learning_rate,
              'subsample': subsample,
              'colsample_bytree': colsample_bytree,
              'min_child_weight': min_child_weight,
              'n_estimators': int(n_estimators),
              'alpha': alpha,
              'eta': eta,
              'eval_metric': 'rmse',
              'tree_method': 'gpu_hist'}
 
    #Cross validating with the specified parameters in 3 folds
    cv_result = xgboost.cv(params, dtrain, nfold=3)
    #Return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

#### n_iter 
* controls how many bayesian optimization steps to perform, more steps = more likely to find a good maximization

#### init_points 
* controls how many steps of random exploration you want to perform to help diversify the exploration space

#### acquisition functions
* decides how to guide the optimization
* acq: {'ucb', 'ei', 'poi'}

In [31]:
# Initial Bayesian Optimization Search
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (4,12),
                                             'gamma': (0.1, 1),
                                             'subsample' : (0.5, 1),
                                             'learning_rate' : (0.0001, 0.01),
                                             'colsample_bytree': (0.7, 1),
                                             'min_child_weight': (4, 10),
                                             'n_estimators':(80, 240),
                                             'alpha': (0.1, 1),
                                             'eta': (0.1, 0.3)
                                            })

xgb_bo.maximize(n_iter=5, init_points=5, acq='ei')

|   iter    |  target   |   alpha   | colsam... |    eta    |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        | -0.2233   |  0.5812   |  0.7408   |  0.241    |  0.7541   |  0.000309 |  11.48    |  4.3      |  145.7    |  0.818    |
|  2        | -0.2233   |  0.358    |  0.9853   |  0.2256   |  0.1545   |  0.008279 |  8.535    |  8.691    |  86.66    |  0.8487   |
|  3        | -0.2233   |  0.8644   |  0.7567   |  0.2783   |  0.5698   |  0.004954 |  11.15    |  4.514    |  223.5    |  0.9694   |
|  4        | -0.2233   |  0.8552   |  0.9783   |  0.1311   |  0.1946   |  0.009649 |  5.148    |  7.246    |  101.2    |  0.6692   |
|  5        | -0.2233   |  0.8987   |  0.7973   |  0.1075   |  0.7394   |  0.009176 |  8.436    |  9.476    |  108.1    |  0.7568   |
|  6        | -0.2233   |  0.1179   |  0.7364   |  0.145    | 

Just for this notebook I kept the number of iterations and initiation points relatively small so it doesn't a long time to run.

In [32]:
# Obtain best parameters from bayesian optimization search

params = xgb_bo.max['params']
#Converting the max_depth and n_estimator values from float to int
params['max_depth']= int(round(params['max_depth']))
params['n_estimators']= int(round(params['n_estimators']))
params

{'alpha': 0.3580124930932794,
 'colsample_bytree': 0.9852618386784621,
 'eta': 0.22559253407743465,
 'gamma': 0.15454714169639497,
 'learning_rate': 0.008279493643181786,
 'max_depth': 9,
 'min_child_weight': 8.690537033017463,
 'n_estimators': 87,
 'subsample': 0.8487037105346853}

In [33]:
#Train XGBRegressor with best params from bayesian optimization
autoenc_X_train = autoencoder.predict(X_train)
xgb_bestparams = xgboost.XGBRegressor(**params, tree_method='gpu_hist').fit(autoenc_X_train, y_train)

[14:55:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


### Model Evaluation

In [34]:
# The models should be scored based on the rank-correlation (spearman) with the target
def numerai_score(y_true, y_pred):
    rank_pred = y_pred.groupby(eras).apply(lambda x: x.rank(pct=True, method="first"))
    return np.corrcoef(y_true, rank_pred)[0,1]

# It can also be convenient while working to evaluate based on the regular (pearson) correlation
def correlation_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]

In [35]:
train_preds = xgb_bestparams.predict(autoenc_X_train)
print('Training Scores')
print('Numerai Score: ', numerai_score(y_train, pd.Series(train_preds)))
print('Correlation Score: ', correlation_score(y_train, pd.Series(train_preds)))
print()

autoenc_X_test = autoencoder.predict(X_test)
test_preds = xgb_bestparams.predict(autoenc_X_test)
print('Testing Scores')
print('Numerai Score: ', numerai_score(y_test, pd.Series(test_preds)))
print('Correlation Score: ', correlation_score(y_test, pd.Series(test_preds)))
print()

autoenc_validation = autoencoder.predict(valid_df[features])
validation_preds = xgb_bestparams.predict(autoenc_validation)
print('Validation Scores')
print('Numerai Score: ', numerai_score(valid_df[target], pd.Series(validation_preds)))
print('Correlation Score: ', correlation_score(valid_df[target], pd.Series(validation_preds)))

Training Scores
Numerai Score:  0.41326823670909
Correlation Score:  0.4638529633347826

Testing Scores
Numerai Score:  0.03111450341716244
Correlation Score:  0.0314354017677558

Validation Scores
Numerai Score:  0.015602395208498848
Correlation Score:  0.017801418534472258


The model clearly overfits the training data because the numerai and correlation scores of the testing and validation sets drop off significantly from the training set scores. This means that the model will most likely not generalize well to the live tournament data. This model might perform well in a couple of rounds, but over time I predict it will generally underperform. 

Personally, the best models I have made that are performing well on the live tournament typically have validation scores between 0.03-0.05 and slightly overfit the training data. 

### Predicting the Live Tournament Data
* Recall we read in the tournament data at the beginning of the notebook under tourney_df
* Make sure the format is correct (id, prediction)

In [24]:
# Apply autoencoder for dimensionality reduction to the tournament data
autoenc_tourney = autoencoder.predict(tourney_df[features])

# Use our best xgboost model from bayesian optimization to predict autoencoded tournament data
# Avoid RAM issues by splitting tourney data
tourney_preds_1 = xgb_bestparams.predict(autoenc_tourney[:1000000])
tourney_preds_2 = xgb_bestparams.predict(autoenc_tourney[1000000:])
tourney_preds = np.concatenate((tourney_preds_1, tourney_preds_2))

df = pd.DataFrame()
df['id'] = tourney_ids
df['prediction'] = tourney_preds
print("Current round: ", current_round)
df.info()

Current round:  260
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1698481 entries, 0 to 1698480
Data columns (total 2 columns):
 #   Column      Dtype  
---  ------      -----  
 0   id          object 
 1   prediction  float32
dtypes: float32(1), object(1)
memory usage: 19.4+ MB


In [25]:
df.head()

,id,prediction
0,n0003aa52cab36c2,0.496543
1,n000920ed083903f,0.492989
2,n0038e640522c4a6,0.518021
3,n004ac94a87dc54b,0.496860
4,n0052fe97ea0c05f,0.496778


In [26]:
# Save as csv file you can upload on the numerai website
df.to_csv(f'round{current_round}_autoenc_xgb_bayesopt_predictions.csv', index=False)

### Documentation and References

Numerapi: https://github.com/numerai/numerapi

Numerai Examples: https://github.com/numerai/example-scripts

Bayesian Optimization: https://github.com/fmfn/BayesianOptimization

XGBoost: https://github.com/dmlc/xgboost/tree/master/demo/guide-python